In [303]:
from pathlib import Path
from medkit.core.text import TextDocument
from medkit.text.segmentation import SentenceTokenizer
from medkit.text.ner import RegexpMatcher, RegexpMatcherRule , RegexpMatcherNormalization
from medkit.text.context import NegationDetector, NegationDetectorRule
from medkit.text.segmentation import SyntagmaTokenizer
from medkit.text.context import FamilyDetector
from unidecode import unidecode
import pandas as pd
import os
import re

In [327]:
import pandas as pd

# Modifier les options d'affichage pour afficher l'intégralité du dataframe
pd.set_option('display.max_columns', None)  # Afficher toutes les colonnes
pd.set_option('display.max_rows', None)  # Afficher toutes les lignes


## PREPROCESSING

In [304]:
"""
La fonction preprocess_text effectue le prétraitement d'un texte en remplaçant certains caractères spéciaux 
par leur équivalent en ASCII. Elle convertit le texte donné en entrée en ASCII et normalise les espaces en remplaçant les 
espaces multiples par un seul espace. La fonction renvoie Le texte prétraité.

"""
def preprocess_text(text):
    # Convertir les caractères spéciaux spécifiques avant la conversion en ASCII
    text = re.sub(r'n°', 'numero', text)  # Remplace "n°" par "numero"
    text = re.sub(r'/d°', 'deg', text)  # Remplace "/d°" par "deg"

    # Convertir le texte en ASCII
    ascii_text = unidecode(text)  # Convertit les caractères Unicode en ASCII

    # Normaliser les espaces en remplaçant les espaces multiples par un seul espace
    text = re.sub(r'\s+', ' ', ascii_text)  # Remplace plusieurs espaces par un seul espace

    return text  # Retourne le texte prétraité en ASCII

## DECOUPAGE EN SENTENCES

In [305]:
"""
La fonction splitting_doc_sentences prend un document en entrée et utilise un tokenizer de phrases pour segmenter le document
en phrases individuelles, en utilisant des caractères de ponctuation tels que ".", "?", et "!". Les phrases segmentées
sont renvoyées en sortie de la fonction.

"""

def splitting_doc_sentences(doc):
    sent_tokenizer = SentenceTokenizer(
        output_label="sentence",
        punct_chars=[".", "?", "!"],
    )
    sentences = sent_tokenizer.run([doc.raw_segment])
    return sentences

## DECOUPAGE EN SYNTAGMAS

In [306]:
"""
La fonction splitting_doc_syntagmas prend des phrases (sentences) en entrée et utilise un tokenizer 
de syntagmes pour segmenter à nouveau les sentences en utilisant en utilisant les séparateurs tels que
"mais" et "et".. Les syntagmas sont renvoyées en sortie de la fonction.

"""
def splitting_doc_syntagmas(sentences):
    synt_tokenizer = SyntagmaTokenizer(
        output_label="sentence",
        separators=[r"\bmais\b", r"\bet\b"],
    )
    syntagmas = synt_tokenizer.run(sentences)
    return syntagmas

In [307]:
## RECHERCHE ENTITÉ

In [308]:
"""
La fonction finding_entities_(fumeur/alcool/situation_familiale) prend en entrée une liste de phrases (sentences) et utilise
des règles de correspondance basées sur des expressions régulières pour extraire des
entités spécifiques du texte. Les entités recherchées sont liées au tabac, à l'alcool, 
au statut marital respectivement. Elle renvoie les entités extraites et les règles 
regex qui ont été utilisées.
"""
def entities_rules_tabac():
    regexp_rules = [

        ## REGEX TABAC
        RegexpMatcherRule(regexp=r"\bcigare(tte)?[s]?\b", label="statut_tabagisme"),
        RegexpMatcherRule(regexp=r"\bfume(e)?[s]?\b", label="statut_tabagisme"),
        RegexpMatcherRule(regexp=r"\btaba(c|gisme|gique)\b", label="statut_tabagisme"),
        RegexpMatcherRule(
        regexp=r"fumeur|fumeuse|paquet.{0,5}?ann[ée].?",
        label="statut_tabagisme",
        unicode_sensitive=True),
    ]
        
    return regexp_rules

In [309]:
"""
La fonction finding_entities prend en entrée une liste de phrases (sentences) et utilise
des règles de correspondance basées sur des expressions régulières pour extraire des
entités spécifiques du texte. Les entités recherchées sont liées au tabac, à l'alcool, 
au statut marital et aux antécédents. Elle renvoie les entités extraites et les règles 
regex qui ont été utilisées.
"""
def entities_rules_alcool():

    
    regexp_rules= [

        ## REGEX ALCOOL
        RegexpMatcherRule(regexp=r"\balcool\b", label="statut_alcool"),
        RegexpMatcherRule(regexp=r"\bboit\b", label="statut_alcool"),
        RegexpMatcherRule(regexp=r"\balcoolique\b", label="statut_alcool"),
        RegexpMatcherRule(regexp=r"\bdependance\s*alcool\b", label="statut_alcool"),
        RegexpMatcherRule(regexp=r"\balcoolisme\b", label="statut_alcool"),

    ]

        
    return regexp_rules

In [310]:
"""
La fonction finding_entities prend en entrée une liste de phrases (sentences) et utilise
des règles de correspondance basées sur des expressions régulières pour extraire des
entités spécifiques du texte. Les entités recherchées sont liées au tabac, à l'alcool, 
au statut marital et aux antécédents. Elle renvoie les entités extraites et les règles 
regex qui ont été utilisées.
"""
def entities_rules_situation_familiale():
    

    regexp_rules= [

        ## REGEX SITUATION FAMILIAL
        RegexpMatcherRule(regexp=r"\bmarie[e]?\b", label="statut_marital"),
        RegexpMatcherRule(regexp=r"\bcelibataire\b", label="statut_marital"),
        RegexpMatcherRule(regexp=r"\bdivorce[e]?\b", label="statut_marital"),
        RegexpMatcherRule(regexp=r"\bveuf\b", label="statut_marital"),
        RegexpMatcherRule(regexp=r"\bveuve\b", label="statut_marital"),
        RegexpMatcherRule(regexp=r"\bpacse[e][s]?\b", label="statut_marital"),
        RegexpMatcherRule(regexp=r"\bconcubinage\b", label="statut_marital"),
        RegexpMatcherRule(regexp=r"\b(vit|habite)\sseul(e)?\b", label="statut_marital"),

    ]

        
    return regexp_rules

## DETECTION NEGATION

In [311]:
"""
La fonction detecting_negation_(fumeur/alcool/situation_familiale) crée un détecteur de négation 
qui utilise des règles basées sur des expressions régulières pour repérer les indicateurs de négation 
dans le texte. Les règlescouvrent les domaines du du tabagisme, de l'alcool et du statut marital (respectivement)
Le détecteur de négation est renvoyé en sortie de la fonction.
"""
def negation_rules_tabac():
    
    neg_rules = [       
        
    NegationDetectorRule(regexp=r"\bne\s*(semble|consomme|prend)\s*pas"),
    NegationDetectorRule(regexp=r"jamais"),
    NegationDetectorRule(regexp=r"arret"),
    NegationDetectorRule(regexp=r"ancien"),
    NegationDetectorRule(regexp=r"\bnon\s*tabagi(que|sme)"),
    NegationDetectorRule(regexp=r"\b(ni|non)\/sfumeu(r|se)"),
    NegationDetectorRule(regexp=r"\bne/s*fume/s*pas\b"),
    NegationDetectorRule(regexp=r"non\\s+fumeur|tabac\\s+non|tabagisme\\s+non|Tabac\\s*[=:]?\\s*0|tabagi(sm|qu)e\\s+sevr|(pas|ni|ou)\\s+de\\s+(consommation\\s+de\\s+)?taba|pas\\s+d\\'intoxication\\s+tabagi|0 tabac",
                        unicode_sensitive=True),
    
    ]
        
    return neg_rules

In [312]:
def negation_rules_alcool():
    
    neg_rules = [
        
        NegationDetectorRule(regexp=r"ne\s*boit\s*pas"),
        NegationDetectorRule(regexp=r"\bne/s*consomme/s*pas\b"),
        NegationDetectorRule(regexp=r"\bni/s*alcool\b"),
    ]
    
    return neg_rules

In [313]:
def negation_rules_situation_familiale():
    
    neg_rules = [

        NegationDetectorRule(regexp=r"\bn'est pas\b"),
        NegationDetectorRule(regexp=r"\bne vit pas\b"),
        NegationDetectorRule(regexp=r"\bn'habite pas\b"),
        NegationDetectorRule(regexp=r"\bni\b"),
    ]
    
    return neg_rules

## EXTRACTION STATUT TABAC

## EXTRACTION STATUT ALCOOL

In [314]:
"""
La fonction statut_extraction extrait les informations de statut d'alcool à partir d'un dictionnaire dico en
parcourant les  annotations, en calculant les proportions et en choisissant les statuts appropriés. 
Elle renvoie les statuts extraits.
"""

def statut_extraction_alcool(doc):

    ## Initialisation
    statut_alcool = "UNKNOWN"

    # Nombre d'entité trouvé
    n_oui_alcool = 0
    n_non_alcool = 0
  
    # Proportion calculée à partir du nombre d'entité trouvé
    p_neg_alcool = 0
    p_pos_alcool = 0

    # On parcourt le dico pour analyser chaque annotation trouvé
    for ann in doc.anns:

        # On initialise les valeurs par défaut à None
        value_is_negated = False
        value_other_detected = False
        for attr in ann.attrs:
            if attr.label== "is_negated":
                # On récupère la valeur is_negated 
                value_is_negated = attr.value
                
            if attr.label== "other_detected":
                # On récupère la valeur other_detected
                value_other_detected = attr.value

            # Si l'entitée trouvée ne concerne pas le patient donc other_detected == True, 
            # on passe directement à l'annotation suivante
            if value_other_detected:
                #print(f"Texte : {ann['text']}, Is_negated : {value_is_negated}, other_detected : {value_other_detected}")
                continue

            # STATUT TABAGISME
            if ann.label== "statut_alcoolique":
                #print(f"Texte : {ann['text']}, Is_negated : {value_is_negated}, other_detected : {value_other_detected}")
                # Si c'est une négation, on incrémente n_non_tabac de 1
                if value_is_negated:
                    n_non_alcool += 1
                # Si ce n'est pas une négation, on incrémente n_oui_tabac de 1
                else:
                    n_oui_alcool += 1

    if len(doc.anns) != 0:        
        ### Porportion TABAC  
        p_neg_alcool = n_non_alcool/len(doc.anns)
        p_pos_alcool = n_oui_alcool/len(doc.anns)

    ## On choisit un statut en fonction des proportions calculées 
    if p_pos_alcool > p_neg_alcool:
        statut_alcool = "ALCOOLIQUE"

    if p_pos_alcool < p_neg_alcool:
        statut_alcool = "NON-ALCOOLIQUE"


    return statut_alcool

## EXTRACTION SITUATION FAMILIALE

In [315]:
def statut_extraction_situation_familiale(doc):

    ## Initialisation
    statut_familial = "UNKNOWN"
    normalized_statut = "UNKNOWN"

    # On parcourt le dico pour analyser chaque annotation trouvé
    for ann in doc.anns:

        # On initialise les valeurs par défaut à None
        value_is_negated = False
        value_other_detected = False
        for attr in ann.attrs:
            if attr.label== "is_negated":
                # On récupère la valeur is_negated 
                value_is_negated = attr.value
                
            if attr.label== "other_detected":
                # On récupère la valeur other_detected
                value_other_detected = attr.value

            # Si l'entitée trouvée ne concerne pas le patient donc other_detected == True, 
            # on passe directement à l'annotation suivante
            if value_other_detected:
                #print(f"Texte : {ann['text']}, Is_negated : {value_is_negated}, other_detected : {value_other_detected}")
                continue

            # STATUT FAMILIALE
            if ann.label== "statut_marital":
                # Si il n'a pas de négation
                if not value_is_negated:
                    statut_marital = ann.text.upper()
                    
                    ## NORMALISATION: Seul, pas seul et inconnu
                    if re.search(r"\bmarie[e]?\b", statut_marital):
                        normalized_statut = "Pas seul"
                    elif re.search(r"\bcelibataire\b", statut_marital):
                        normalized_statut = "Seul"
                    elif re.search(r"\bdivorce[e]?\b", statut_marital):
                        normalized_statut = "Seul"
                    elif re.search(r"\bveuf\b", statut_marital):
                        normalized_statut = "Seul"
                    elif re.search(r"\bveuve\b", statut_marital):
                        normalized_statut = "Seul"
                    elif re.search(r"\bpacse[e][s]?\b", statut_marital):
                        normalized_statut = "Pas seul"
                    elif re.search(r"\bconcubinage\b", statut_marital):
                        normalized_statut = "Pas seul"
                    elif re.search(r"\b(vit|habite)\sseul(e)?\b", statut_marital):
                        normalized_statut = "Seul"
                    else:
                        normalized_statut = statut_marital  # Conservez le statut non reconnu tel quel
                
                # Si il y'a une négation, on inverse le statut trouvé
                else:
                    if normalized_statut == "Seul":
                        normalized_statut == "Pas seul"
                    else:
                        normalized_statut == "Seul"
            
    return normalized_statut  

In [322]:
"""
La fonction statut_extraction extrait les informations de statut de tabagisme, 
statut marital et statut d'alcool à partir d'un dictionnaire dico en parcourant les 
annotations, en calculant les proportions et en choisissant les statuts appropriés. 
Elle renvoie les statuts extraits.
"""

def statut_extraction_tabac(doc):

    ## Initialisation
    statut_tabagisme = "UNKNOWN"

    # Nombre d'entités trouvées
    n_oui_tabac = 0
    n_non_tabac = 0

    # Proportions calculées à partir du nombre d'entités trouvées
    p_neg_tabac = 0
    p_pos_tabac = 0

    # Parcours des annotations pour analyser chaque annotation trouvée
    for ann in doc.anns:
        if ann.label == "statut_tabagisme":
            for attr in ann.attrs:
                if attr.label == "is_negated":
                    value_is_negated = attr.value
                elif attr.label == "other_detected":
                    value_other_detected = attr.value

            # Si l'entité trouvée ne concerne pas le patient (other_detected == True),
            # on passe à l'annotation suivante
            if value_other_detected == True:
                continue
            else:
                # Si c'est une négation, on incrémente n_non_tabac de 1
                if value_is_negated == True:
                    n_non_tabac += 1
                # Sinon, on incrémente n_oui_tabac de 1
                else:
                    n_oui_tabac += 1

    if len(doc.anns) != 0:
        # Calcul des proportions de tabagisme
        p_neg_tabac = n_non_tabac / len(doc.anns)
        p_pos_tabac = n_oui_tabac / len(doc.anns)

    ## Choix du statut en fonction des proportions calculées
    if p_pos_tabac > p_neg_tabac:
        statut_tabagisme = "FUMEUR"
    elif p_pos_tabac < p_neg_tabac:
        statut_tabagisme = "NON-FUMEUR"


    return statut_tabagisme

## RECUPERATION DES CAS CLINIQUES DANS UN DICO{NOM_FICHIER}=CAS_CLINIQUE

In [317]:
## Meme fonction mais renvoie un dico[nom fichier]= cas clinique

def clinical_case_recovery(output_folder):
    # On récupère tous les fichiers texte dans le dossier
    txt_files = [f for f in os.listdir(output_folder) if f.endswith('.txt')]

    # On trie les fichiers par ordre alphabétique.
    txt_files_sorted = sorted(txt_files)

    textes = {}  # dictionnaire de tous les cas cliniques

    # On ouvre et on extrait les textes dans textes
    for i in range(500):
        file_path = os.path.join(output_folder, txt_files_sorted[i])
        with open(file_path, 'r') as f:
            text = f.read()
        textes[txt_files_sorted[i]] = text
    return textes


# On exécute les fonctions sur les cas cliniques

In [328]:
# TEST ON REAL CLINICAL CASES
output_folder = "clinical_case1"
textes = []
textes = clinical_case_recovery(output_folder)
# Listes pour stocker les données
data_fumeur = []
data_non_fumeur = []
data_unknown = []



# On parcourt clé (nom fichier), valeur (cas clinique) du dico textes
for fichier, cinical_case in textes.items():

    # PREPROCESSING THE TEXT (CLINICAL CASE)
    cinical_case = preprocess_text(cinical_case)

    # LOADING A TEXT DOCUMENT 
    doc = TextDocument(text=cinical_case)

    # SPLITTING A DOCUMENT IN SENTENCES
    sentences = splitting_doc_sentences(doc)
    
    # SPLITTING A DOCUMENT IN SYNTAGMAS
    syntagmas = splitting_doc_syntagmas(sentences)

    # FINDING ENTITIES
    regexp_rules_tabac = entities_rules_tabac()
    regexp_rules_alcool = entities_rules_alcool()
    regexp_rules_situation_familiale = entities_rules_situation_familiale()

    
    # Utilisation des règles converties avec RegexpMatcher
    regexp_matcher_tabac = RegexpMatcher(rules=regexp_rules_tabac)
    regexp_matcher_alcool = RegexpMatcher(rules=regexp_rules_alcool)
    regexp_matcher_situation_familiale = RegexpMatcher(rules=regexp_rules_situation_familiale)

    # DETECTING NEGATION
    neg_rules_tabac = negation_rules_tabac()
    neg_rules_alcool = negation_rules_alcool()
    neg_rules_situation_familiale = negation_rules_situation_familiale()
    
    # Création de l'objet neg detector
    neg_detector_tabac = NegationDetector(output_label="is_negated", rules=neg_rules_tabac)
    neg_detector_alcool = NegationDetector(output_label="is_negated", rules=neg_rules_alcool)
    neg_detector_situation_familiale = NegationDetector(output_label="is_negated", rules=neg_rules_situation_familiale)

    # RUN NEG DETECTOR SUR LES SYNTAGMAS
    neg_detector_tabac.run(syntagmas)
    neg_detector_alcool.run(syntagmas)
    neg_detector_situation_familiale.run(syntagmas)

    # RUN FAMILY DETECTOR
    family_detector = FamilyDetector(output_label='other_detected')
    family_detector.run(syntagmas)

    # CREATION OF ENTITIES
    regexp_matcher = RegexpMatcher(rules=regexp_rules_tabac + regexp_rules_alcool + regexp_rules_situation_familiale,
                                   attrs_to_copy=["is_negated", "other_detected"])

    entities = regexp_matcher.run(syntagmas)

    # AUGMENTING A DOCUMENT
    for entity in entities:
        doc.anns.add((entity))

    # STATUT EXTRACTION
    statut_tabagisme = statut_extraction_tabac(doc)
    statut_alcool = statut_extraction_alcool(doc)
    statut_familial = statut_extraction_situation_familiale(doc)
    
    if statut_tabagisme == "FUMEUR":
        print(cinical_case)
        print(statut_tabagisme)
        print("\n")

    # Remplissage de data
    
    if statut_tabagisme == "FUMEUR":
        data_fumeur.append([fichier, cinical_case, statut_tabagisme, statut_alcool, statut_familial])
    elif statut_tabagisme == "NON-FUMEUR":
        data_non_fumeur.append([fichier, cinical_case, statut_tabagisme, statut_alcool, statut_familial])
    else:
        if len(data_unknown) < 10:
            data_unknown.append([fichier, cinical_case, statut_tabagisme, statut_alcool, statut_familial])

            
df_fumeur = pd.DataFrame(data_fumeur, columns=["nom fichier", "cas clinique","statut tabagisme","statut alcool", "statut familial"])


df_non_fumeur = pd.DataFrame(data_non_fumeur, columns=["nom fichier", "cas clinique","statut tabagisme", "statut alcool", "statut familial"])
df_unknown = pd.DataFrame(data_unknown, columns=["nom fichier", "cas clinique","statut tabagisme", "statut alcool", "statut familial"])
combined_df = pd.concat([df_fumeur, df_non_fumeur, df_unknown], ignore_index=True)
# On mélange 
combined_df = combined_df.sample(frac=1, random_state=42)
combined_df.shape
combined_df['statut tabagisme'].value_counts()

Patient age de 45 ans, tabagique chronique, admis dans le service pour exploration d'une adenopathie laterocervicale droite, apparue depuis 4 mois et augmentant progressivement de taille dans un contexte d'amaigrissement non chiffre, accompagnee d'obstruction nasale et d'hypoacousie homolaterales. L'examen clinique a retrouve une adenopathie sous digastrique droite, ferme, indolore, et mesurant 3cm de diametre, une otite seromuqueuse droite. La nasofibroscopie a mis en evidence un bourgeon tumoral de la paroi postero laterale droite du cavum. Le scanner cervical a montre un processus tumoral de la paroi posterieure du cavum avec respect de la graisse parapharyngee et presence d'adenopathies retroparyngees. Des biopsies multiples du cavum ont ete realisees et l'etude anatomopathologique a montre la presence de granulomes epithelio-giganto-cellulaires avec necrose caseeuse. Le bilan biologique a montre une IDR positive et un syndrome inflammatoire manifeste, la radiographie pulmonaire et

Mr HL age de 58 ans, tabagique est admis aux urgences pour des douleurs thoraciques retrosternale irradiant vers l'epaule gauche survenues lors d'un effort physique, accompagnees de nausees et de sueurs. L'examen clinique a retrouve un patient apyretique en bon etat hemodynamique avec assourdissement des bruits du c'ur a l'auscultation cardiaque et une ischemie sous epicardique apico laterale sur l'ECG. La radiographie thoracique a montre un aspect legerement globuleux de l'arc inferieur gauche. Le bilan biologique standard etait sans particularite en dehors d'une hypereosinophilie a 15%. Le patient fut admis en cardiologie pour angor d'effort. L'echocardiographie trans thoracique a objective la presence d'un processus tumoral kystique multivesiculaire siegeant au niveau de la paroi laterale du ventricule gauche occupant toute l'epaisseur du segment moyen et apical avec trouble de la contractilite a ce niveau et un epanchement pericardique de moyenne abondance. Le scanner a mis en evid

Une femme agee de 51 ans, non tabagique, suivi pour un diabete de type 2 depuis 2 ans, sous antidiabetiques oraux, hospitalisee pour une fievre prolongee evoluant depuis un mois, des sueurs nocturnes. Elle avait une anorexie et un amaigrissement chiffre a 7 Kg en deux mois. Il n'y avait pas de notion d'antecedents personnels de tuberculose ni de contage tuberculeux. Elle a ete vaccinee par le vaccin BCG et n'avait pas recu de traitement corticoide ou immunosuppresseur. Par ailleurs, elle se plaignait d'une dysphonie recente. L'examen physique a revele une fievre de 38.5 deg C. Il n'y avait pas d'adenopathie, ni d'hepato-splenomegalie. Les examens cardiaques, respiratoires, neurologiques, musculaires, ophtalmologiques et osteo-articulaire etaient sans anomalies. Plusieurs series d'hemocultures realisees en urgence etaient negatives. L'examen biologique a revele un syndrome inflammatoire avec une vitesse de sedimentation a 70 mm, une C reactive proteine a 35 mg/l et une hypergammaglobuli

Une femme agee de 52 ans, non tabagique, ayant un diabete de type 2 a ete hospitalisee pour une pneumopathie infectieuse. Cette patiente presentait depuis 2 ans des episodes recidivants d'infections respiratoires traites en ambulatoire. L'examen physique a trouve une fievre a 38oc et un foyer de rales crepitants de la base pulmonaire droite. La radiographie du thorax a mis en evidence une opacite alveolaire du lobe moyen. La biologie a revele un syndrome inflammatoire avec des globules blancs a 7400/mm 3, une anemie avec une hemoglobinemie a 10,3 g/dl, une C-reactive proteine (CRP) a 48 mg/l et une vitesse de sedimentation (VS) a 50. L'examen cytobacteriologique des crachats a isole un Streptocoque Pneumoniae. Le diagnostic de pneumopathie bacterienne a Streptocoque Pneumoniae a ete retenu et la patiente a ete traitee par antibiotherapie adaptee. L'evolution etait favorable, marquee par une defervescence thermique et une disparition du syndrome inflammatoire biologique. Sur le plan rad

Un patient de 62 ans, tabagique chronique, sans autres antecedents pathologiques particuliers, consulte pour une toux seche evoluant depuis 4 mois associee a une dyspnee d'effort d'aggravation progressive. La radiographie du thorax a montre une opacite lobaire inferieure droite. La tomodensitometrique thoracique faite par la suite a objective un processus tumorale du lobe inferieure droit de 48x42 mm localise en paravertebrale droit. La bronchoscopie a mis en evidence un aspect inflammatoire de 1 er degre des bronches droites sans tumeur visible. Il a ete decide de faire une biopsie pulmonaire scannoguidee. Cette derniere a montre une proliferation tumorale a cellules rondes a differenciation majoritairement plasmocytoide. L'etude immunohistochimique a montre une positivite des anticorps anti CD138, EMA et anti KI67 et une negativite des anti chromogranine et synaptophysine. Biologiquement, on a note une augmentation des gammaglobulines en rapport avec l'existence d'un pic monoclonal d

Nous rapportons le cas d'un patient age de 45 ans sans antecedents pathologiques particuliers, admis en unite de soins intensifs pour reanimation post-operatoire d'un triple pontage aorto-coronaire. Les explorations qui ont ete realisees en preoperatoire ont montre une bonne fonction ventriculaire gauche systolique et diastolique a l'echographie cardiaque. La coronarographie a objective une stenose significative du tronc commun gauche, une double stenoses serrees de l'IVA distale, une stenose tres serree de la CDII et une stenose serree de la CD III et le doppler des troncs supra aortiques etait sans anomalie. Dans ses habitudes, nous avons note l'absence stricte de consommation d'alcool, une consommation de 40 cigarettes par jour avec une intoxication tabagique evaluee a 50 paquets.annee. Les durees de la chirurgie, de la CEC et du clampage aortique ont ete respectivement de 240 min, 100 min et 60 min. Le patient a ete extube a H2 post-operatoire (PO) et sevre des catecholamines a j1P

statut tabagisme
FUMEUR        25
UNKNOWN       10
NON-FUMEUR     2
Name: count, dtype: int64